# 카카오 쇼핑 크롤링

목표
- 정적 페이지, 동적 페이지 구분하기
- `스크롤`을 활용하여 300개의 상품 정보 추출
- 상품 정보
  1. 상품명
  2. 상페 상품 페이지 링크
  3. 판매가격
  4. 판매처
  5. 배송정보 e.g. 무료배송, 유료배송
  6. 상품 이미지

chrome settings -> preferences -> disable javascript를 통해 동적인지 정적인지 확인해보니 `동적`이다.


In [89]:
from dataclasses import dataclass
import random
from time import sleep
from typing import ClassVar, Optional

import pandas as pd
from urllib.parse import urljoin
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.remote.webelement import WebElement
from bs4 import BeautifulSoup


KAKAO_SHOPPING_ROOT = "https://shoppinghow.kakao.com"
KAKAO_SHOPPING_PATH = "/siso/p/sale/mall/talkstore"
EXCEL_FILE_PATH = "./outputs/kakao-shopping.xlsx"

@dataclass
class DriverController:
  DEFAULT_DELAY: ClassVar[int] = 5 # 클래스 상수 ClassVar를 사용안하면 에러 발생
  RANDOM_SLEEP_MIN: ClassVar[float] = 0.8
  RANDOM_SLEEP_MAX: ClassVar[float] = 1.5
  driver: webdriver.Chrome
  start_url: Optional[str] = None

  # __init__ 후에 바로 실행되는 special method
  def __post_init__(self) -> None:
    if self.start_url:
      self.driver.get(self.start_url)

      WebDriverWait(self.driver, 10).until(
        lambda d: d.execute_script("return document.readyState") == "complete"
      )

  # 기본적으로 명시적 대기 기반 요소 찾기 
  def find_element(
    self, 
    selector: str, 
    driver_selector = By.CSS_SELECTOR, 
    delay = DEFAULT_DELAY
  ) -> WebElement:
    return WebDriverWait(self.driver, delay).until(
      EC.presence_of_element_located((driver_selector, selector))
    )
  
  def click_element(self, selector: str) -> None:
    element = self.find_element(selector)
    element.click()
    self._random_sleep()
  
  def key_input(self, selector: str, keys: list[str]) -> None:
    element = self.find_element(selector)
    element.send_keys(*keys)
    self._random_sleep()
  
  # 동일한 term으로 실행시 차단될 수 있으니 random한 값으로 sleep 수행
  def _random_sleep(self) -> None:
    sleep(random.uniform(self.RANDOM_SLEEP_MIN, self.RANDOM_SLEEP_MAX))
  
  
driver = webdriver.Chrome()
dc = DriverController(driver=driver, start_url=urljoin(KAKAO_SHOPPING_ROOT, KAKAO_SHOPPING_PATH))


# 원하는 조건까지 무한 스크롤

In [90]:
# item이 300까지 일 때 까지 scroll
LIMIT_ITEMS_COUNT = 300
items_wrapper_selector = "#SaleSectionProductsWrap > li"

js_script = f"""
itemsLength = document.querySelectorAll(arguments[0]).length;
window.scrollBy(0, 2800);

return itemsLength;
"""

while True:
  items_length = driver.execute_script(js_script, items_wrapper_selector)
  sleep(1)
  
  if items_length >= LIMIT_ITEMS_COUNT: break


# 원하는 정보 추출

In [91]:
page_soup = BeautifulSoup(driver.page_source, "html.parser")
items = page_soup.select(items_wrapper_selector)
result = []

for item in items:
  product_name = item.select_one(".info_name").get_text(strip=True)
  product_link = urljoin(KAKAO_SHOPPING_ROOT, item.select_one("a")["href"])
  product_price = item.select_one(".info_sales").get_text(strip=True).replace(",", "").replace("원", "")
  product_seller = item.select_one(".txt_shop").text
  product_shipping = item.select_one(".info_etc").contents[-1]
  product_image_link = "https:" + item.select_one(".wrap_thumb > img")["src"]
  # print(product_name, product_link, product_price, product_seller, product_shipping, product_image_link, sep="\n")

  result.append([
    product_name, 
    product_link, 
    product_price, 
    product_seller, 
    product_shipping, 
    product_image_link
  ])



# 엑셀에 저장

In [92]:
import pandas as pd


df = pd.DataFrame(result, columns=["상품명", "상품 링크", "가격", "판매처", "배송정보", "상품 이미지"])
df.to_excel(EXCEL_FILE_PATH,  index=False)


# 실제 이미지 저장하기


In [93]:
from io import BytesIO

import requests
from openpyxl import load_workbook
from openpyxl.drawing.image import Image as ExcelImage
from PIL import Image as PILImage


def get_excel_format_image_from_url(url: str, *, width: Optional[int] = 100, height: Optional[int] = 100) -> ExcelImage:
  response = requests.get(url)
  response.raise_for_status()

  image_raw_bytes = response.content
  stored_image_in_buffer = BytesIO(image_raw_bytes)
  image_pil_obj = PILImage.open(stored_image_in_buffer)
  scaled_down_image_pil_obj = image_pil_obj.resize((width, height), PILImage.Resampling.LANCZOS)
  # 제대로 들어왔나 확인
  # scaled_down_image_pil_obj.show()

  # 다시 excel image object롤 변환
  temp_buffer = BytesIO()
  scaled_down_image_pil_obj.save(temp_buffer, format="PNG", optimize=True)
  temp_buffer.seek(0)
  excel_image_obj = ExcelImage(temp_buffer)

  return excel_image_obj

def insert_image_to_cell(ws, column_letter: str, row_index: int, excel_image: ExcelImage) -> None:
  cell_coordinate = f"{column_letter}{row_index}"
  
  # 해당 셀에 이미지 링크가 있으므로 초기화
  ws[cell_coordinate] = None

  # 이미지의 크기에 맞게 cell 크기 조정
  ws.column_dimensions[column_letter].width = excel_image.width / 7
  ws.row_dimensions[row_index].height = excel_image.height * 0.8

  ws.add_image(excel_image, cell_coordinate)



wb = load_workbook(EXCEL_FILE_PATH)
ws = wb.active

column_name_letter_map = {column.value: column.column_letter for column in ws[1]}
image_column_letter = column_name_letter_map["상품 이미지"]

# 상품 이미지에 담겨 있는 상품 이미지 링크를 순회하여 실제 이미지로 추가
for row_index in range(2, ws.max_row + 1):
  image_url = ws[f"{image_column_letter}{row_index}"].value
  excel_image = get_excel_format_image_from_url(image_url)
  
  insert_image_to_cell(
    ws, 
    image_column_letter, 
    row_index, 
    excel_image
  )

wb.save(EXCEL_FILE_PATH)


# 이미지 저장 정리

crawling을 진행하면서 동시에 이미지를 값으로 저장하여 엑셀에 저장을 하려고 했으나, 이미지는 `엑셀` 포맷이 달라 아래와 같이 진행해야 한다.
1. 먼저 이미지 링크를 excel에 저장
2. excel에 존재하는 이미지 링크들을 읽으면서 이미지로 대체

## byte code로 이미지 가져오기

image url에 접근하면 해당 데이터는 image를 `byte code 즉, binary data`로 내려준다.  
이 때 처음에는 text를 통해서 가져왔는데 차이점이 존재한다.

- response.text => 응답이 string type, `encoding 과정에서 값이 깨지는 문제`가 발생
- response.content => string type이 아닌 `byte`로 받아옴

즉, image는 byte로 다루어야 하는데 text로 다룰려고 하니까 오류가 생겼던 것이다.

In [96]:
import requests

IMAGE_URL = "https://shop1.daumcdn.net/thumb/R250x250.q90/?fname=http%3A%2F%2Fshop1.daumcdn.net%2Fshophow%2Fp%2FX19847100344.jpg%3Fut%3D202511"

response = requests.get(IMAGE_URL)
response.raise_for_status()

## 메모리 버퍼에 이미지 byte code 저장하기

메모리 버퍼에 이미지의 byte code를 저장한다.  
왜 메모리 버퍼에 이미지 데이터인 byte code를 저장할까 🤔?  

엑셀에 이미지를 넣을 때 내가 생각한 방법은 두 가지이다.
1. 순회하면서 동시에 이미지 링크에 접근, 데이터를 다운 받아 엑셀에 넣기
2. crawling 과정 순회 때 image를 파일로 저장하여 이를 다시 엑셀을 읽으면서 넣기

여기서 이를 이해하기 위해선 `파일`은 CS측면에서 무엇인지를 제대로 알아야 한다.  
매우 중요한 포인트는 `파일은 디스크에 저장되어 있다`라는 것이 포인트이다.
> 파일이란? 🤔🤔🤔   
> 파일을 정확하게 말하자면 `디스크에 저장되어 있는 OS가 관리하는 데이터 묶음 단위`이다.  
> 내가 이해한 것을 토대로 정의한 것이라 정확하지 않을 수 있지만 맞는 거 같다.

이제 1, 2방법의 차이점을 알아보자. 주요 차이점은 `디스크에 접근하여 가져오냐` 아니면 `메모리 단에서 처리`할 것이냐다.
- 1번 방법: `메모리 버퍼`에 바이트 코드를 저장하므로 디스크에 접근하는 과정이 생략되어 overhead가 적다.
- 2번 방법: 디스크에 접근해야 되므로 overhead가 크다.

이렇게 `메모리 단에서 바이트 코드(현재는 image data)`를 처리하게 해 주는 것이 `BytesIO`이다.
결론을 내리면 BytesIO는 <span style="color: orange;">디스크에 저장되어 있는 파일이 아닌 바이트 코드를 메모리단에서 파일처럼 처리할 수 있게 해주는 역할</span>을 한다.  
그리고 PILImage는 file을 입력으로 받는데 BytesIO를 통해서 해결이 가능한 것이다.



In [97]:
from io import BytesIO

image_raw_bytes = response.content
# print("image_raw_bytes:", image_raw_bytes)
stored_image_in_buffer = BytesIO(image_raw_bytes) # image_file: <_io.BytesIO object at 0x11181ae30> 👉🏻 메모리에 저장되어 있는 것을 확인

## PILImage로 이미지 처리

`stored_image_in_buffer`에는 메모리 버퍼에 이미지 데이터가 byte code로 저장되어 있다.  
이제 `PILImage`를 통해서 해당 데이터를 이미지로 변환하여 처리할 수 있다.  
이 때 PILImage 객체로 변환하는데 당연하지만 `byte image data`를 `python이 알아먹을 수 있는 형태`로 변환해야 하기 때문이다.  

> 💡 PILImage package -> python을 통해 이미지에 대한 조작을 할 수 있게 해주는 패키지

사실 <span style="color: orange;">openpyxl 패키지를 통해 엑셀에 이미지를 바로 주입</span>해도 되지만 공부하는 김에 `PILImage`를 다룬 것이다.

PILImage를 통해 아래 내용을 수행하고자 했다.

- 이미지 크기 변환(이것도 그냥 `openpyxl`에서 할 수 있다.)



In [98]:
from PIL import Image as PILImage

# 메모리 버퍼에 있는 image byte code를 python이 이해하는 형태로 변환
image_pil_obj = PILImage.open(stored_image_in_buffer)
# resize task
scaled_down_image_pil_obj = image_pil_obj.resize((100, 100), PILImage.Resampling.LANCZOS)

## 엑셀에 이미지 저장

엑셀에 저장하기 위해선 `엑셀이 알아먹는 형태`로 이미지를 변환해야 한다. 이는 `PILImage`이 byte code를 python이 알아먹는 형태로 사용하게 해주는 이유와 동일하다.  

엑셀 형태의 이미지로 변환하는 과정은 PILImage 객체 변환 과정과 동일하다.  
즉, 현재 PILIMage를 통해 resize한 데이터를 다시 `openpyxl`을 통해 다시 읽기 위해선 다시 byte code로 변환해야 한다.  


In [99]:
from io import BytesIO

# PILImage를 통해 특정 작업을 수행한 데이터를 메모리 버퍼에 임시적으로 저장하기 위함.
# 이유는 이 임시 저장된 데이터를 다시 openpyxl에서 읽기 위함.
img_temp_memory_buffer = BytesIO()

# PILImage 객체를 byte 형태로 변환하여 위에서 할당한 `img_temp_memory_buffer`에 저장
scaled_down_image_pil_obj.save(img_temp_memory_buffer, format='PNG', optimize=True)

# 메모리 버퍼에 byte code를 쓰면서 포인터가 마지막으로 가 있는데 이를 다시 처음 위치로 지정
# 처음 위치로 지정해야 `openpyxl`에서 처음부터 데이터를 올바르게 읽을 수 있음.
scaled_down_image_pil_obj.seek(0)

## Excel에 이미지 저장

Excel에 이미지를 저장하기 위해 excel이 알아먹는 이미지 형태로 변환하기 위해 `openpyxl`의 `Image` 객체를 사용한다.  

그리고 ws.column_dimensions, ws.row_dimensions에 excel cell의 크기를 조정해주는데 이는 이미지는 `pt`단위이지만 excel은 `font`크기의 단위이기 때문에 `7, 0.75`값을 많이 사용한다고 한다.  

사실 여러번 테스트 해보면서 값을 수정해도 상관없다.(난 위에서 0.8을 사용)

In [100]:
from openpyxl.drawing.image import Image as ExcelImage
from openpyxl import load_workbook

# 메모리 버퍼에 저장되어 있는 image data를 엑셀이 이해하는 포맷인 Image 객체로 변환
excel_img = ExcelImage(img_temp_memory_buffer)

# Excel 파일 열기
wb = load_workbook("./outputs/test.xlsx")
ws = wb.active # 활성 워크시트 가져오기

# 이미지 excel에 저장
# excel_img.width, excel_img.height = 100, 100
ws.column_dimensions['A'].width = excel_img.width / 7
ws.row_dimensions[2].height = excel_img.height * 0.75
ws.add_image(excel_img, "A2")
wb.save("./outputs/test.xlsx")

## 테스트

In [101]:
import pandas as pd
from PIL import Image as PILImage
from openpyxl.drawing.image import Image as ExcelImage
from openpyxl import load_workbook
from io import BytesIO


# ============ excel에 이미지 링크 저장 ============
IMAGE_URL = "https://shop1.daumcdn.net/thumb/R250x250.q90/?fname=http%3A%2F%2Fshop1.daumcdn.net%2Fshophow%2Fp%2FX19847100344.jpg%3Fut%3D202511"
FILE_PATH = "./outputs/test.xlsx"

df = pd.DataFrame([IMAGE_URL])
df.to_excel(FILE_PATH, index=False)
# ================================================

# ============ excel에 실제 이미지 저장 ============
wb = load_workbook(FILE_PATH)
ws = wb.active

response = requests.get(ws["A2"].value)
response.raise_for_status()

image_raw_bytes = response.content
stored_image_in_buffer = BytesIO(image_raw_bytes)
image_pil_obj = PILImage.open(stored_image_in_buffer)
scaled_down_image_pil_obj = image_pil_obj.resize((100, 100), PILImage.Resampling.LANCZOS)

img_temp_memory_buffer = BytesIO()
scaled_down_image_pil_obj.save(img_temp_memory_buffer, format='PNG', optimize=True)
scaled_down_image_pil_obj.seek(0)

excel_img = ExcelImage(img_temp_memory_buffer)
ws.column_dimensions['A'].width = excel_img.width / 7
ws.row_dimensions[2].height = excel_img.height * 0.75
ws.add_image(excel_img, "A2")

wb.save("./outputs/test.xlsx")